- 설치 : pip install langchain-google-community
- 확인은 install 대신 show

In [1]:
from langchain_google_community import GoogleSearchAPIWrapper
import os

In [ ]:
def google_search(query:str) -> str:
    '''
    구글 검색을 통해 필요한 정보를 검색합니다.
    Args:
        query : 검색어
    Return:
        검색 결과 문자열 또는 에러 메세지
    '''
    try:
        # 검색 객체를 생성
        search = GoogleSearchAPIWrapper(google_api_key=os.getenv('GOOGLE_API_KEY'))
        
        # 검색 실행
        result = search.run(query=query)
        return result
    except Exception as e:
        return f'검색 중 오류 발생 : {str(e)}'

In [3]:
query = '2025년 12월에 서울에서 열리는 축제'
response = google_search(query)

In [4]:
print(response)

11월 문화달력 2025-11-03 13:18:44 [문화] 서울시에서 11월에 열리는 문화행사 뉴스레터 공연, 전시, 체험 등 다양한 정보 포함 이번달에 뭐할까? 고민하지 말고 뉴스레터 ... 2025. 10. 1. ... 12일 코엑스 전시장에서 개최되는 '한일축제한마당 2025' 전야제 행사의 일환으로 광림아트센터 장천홀에서 열린다. ... 월 12일(일)에 코엑스 전시장에서 ... 2024. 12. 11. ... 서울콘, 서울라이트 DDP 등도 차례로 열릴 예정. 자세한 내용은 @seoulwinterfesta에서 확인하세요 1️⃣ 서울 빛초롱 축제 🗓️ 2024년 12월 13일 (금) ~ 2025 ... 2025. 3. 6. ... 2025년을 더욱 특별하게 해줄 서울의 축제 일정이 공개됐습니다! 화창한 ... 축제는 5월 10~11일 양일간 노들섬에서 개최된다. 타악기 공연은 ... [세종문화회관] 2025년 12월 누구나 클래식 행복동행석 신청 안내. 2025-11-06 ... 서울시에서 11월에 열리는 문화행사 뉴스레터 공연, 전시, 체험 등 다양한 정보 ... 2025. 4. 25. ... [중랑소식 독자 퀴즈] · 중랑 · 5월 16일(금)부터 24일(토)까지 · 중랑장미공원 일대에서 열립니다. · 올해 열리는 중랑 · 몇 회일까요? (2면 참조) · ① 제15회 ... ※ 코엑스에서 개최되는 모든 행사들이 노출되는 것은 아니며, 주최자(임차인)가 요청하였거나, 일부 행사의 성격에 따라 노출되지 않을 수 있습니다. 2025. 1. 12. ... 소개: 서래섬에 만개한 유채꽃을 배경으로 열리는 봄 축제입니다. 유채 ... 소개: 가을을 맞아 서울식물원에서 열리는 특별 행사로, 다양한 식물 ... 전국 방방곡곡에서 열리는 축제 정보를 소개합니다. 전국의 다채로운 축제와 함께 행복하고 즐거운 여행 되세요! ※ 공간별로 아래 탭을 클릭하여 월간 일정을 확인하세요. 음악당; 오페라하우스; 미술관·박물관. 이전 달 2025 년 11 월 ...


In [6]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AIMessage
from langchain_core.tools import tool
import logging

In [7]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

load_dotenv()

True

In [ ]:
def setup_search_agent():
    """
    검색 조회 에이전트를 설정합니다.
    
    Returns:
        tuple: (llm_with_tools, tools_dict)
    """
    llm = ChatOpenAI(model="gpt-4.1-mini")
    tools = [google_search]
    
    # 도구 설정된 LLM
    llm_with_tools = llm.bind_tools(tools)
    tools_dict = {
        'google_search': google_search
    }
    
    logger.info("검색 에이전트 설정 완료")
    return llm_with_tools, tools_dict

In [10]:
def execute_tools(llm_with_tools, tools_dict, response):
    """
    LLM이 호출한 도구들을 실행하고 ToolMessage로 응답합니다.
    
    Args:
        llm_with_tools: 도구가 바인딩된 LLM
        tools_dict: 도구 이름 - 함수 매핑 딕셔너리
        response: LLM의 응답 (tool_calls 포함)
    
    Returns:
        list: ToolMessage가 포함된 메시지 리스트
    """
    messages = []  # ← 새로 생성 (기존 누락된 부분)

    # tool_calls 유효성 검사
    tool_calls = getattr(response, "tool_calls", None)
    if not tool_calls:
        logger.warning("도구 호출이 없습니다.")
        return messages

    logger.info(f"도구 호출 발견: {len(tool_calls)}개")

    for tool_call in tool_calls:
        try:
            # 구조 분해
            tool_name = tool_call.get("name")
            tool_args = tool_call.get("args", {})
            tool_call_id = tool_call.get("id")

            logger.info(f"도구 실행 요청 ▶ {tool_name}")
            logger.info(f"  - ID: {tool_call_id}")
            logger.info(f"  - Args: {tool_args}")

            # 등록된 도구 확인
            if tool_name not in tools_dict:
                logger.error(f"❌ 알 수 없는 도구: {tool_name}")
                messages.append(
                    ToolMessage(
                        tool_call_id=tool_call_id,
                        name=tool_name,
                        content=f"알 수 없는 도구입니다: {tool_name}"
                    )
                )
                continue

            selected_tool = tools_dict[tool_name]

            # 함수형 도구 / LangChain Tool 모두 지원
            if hasattr(selected_tool, "invoke"):
                tool_result = selected_tool.invoke(tool_args)
            else:
                tool_result = selected_tool(**tool_args)

            result_preview = str(tool_result)
            if len(result_preview) > 200:
                result_preview = result_preview[:200] + "..."
            logger.info(f"✅ 도구 실행 결과: {result_preview}")

            # ToolMessage 구성
            tool_message = ToolMessage(
                tool_call_id=tool_call_id,
                name=tool_name,
                content=str(tool_result)
            )
            messages.append(tool_message)

        except Exception as e:
            logger.error(f"⚠️ 도구 실행 중 오류: {e}")
            messages.append(
                ToolMessage(
                    tool_call_id=tool_call.get("id", "unknown"),
                    name=tool_call.get("name", "unknown"),
                    content=f"도구 실행 중 오류가 발생했습니다: {str(e)}"
                )
            )

    return messages

In [13]:
def ask_search(user_question: str) -> str:
    """
    사용자 질문에 대해 검색 도구를 활용해 답변을 제공합니다.
    
    Args:
        user_question: 사용자의 질문 (예: "Python 프로그래밍이 뭐야?")
    
    Returns:
        LLM이 생성한 최종 답변
    """
    logger.info(f"\n{'='*60}")
    logger.info(f"🔵 사용자 질문: {user_question}")
    logger.info('='*60)
    
    try:
        # 1. 검색 에이전트 설정
        llm_with_tools, tools_dict = setup_search_agent()
        
        # 2. 초기 메시지 구성
        messages = [
            SystemMessage(
                "너는 친절한 검색 정보 제공 봇이야. "
                "사용자가 요청한 정보를 정확하고 친절하게 알려줘. "
                "항상 한국어로 답변하고, 필요한 경우 결과를 요약해서 보기 좋게 표현해."
            ),
            HumanMessage(user_question),
        ]
        logger.info("메시지 초기화 완료")
        
        while True:
            # 3. LLM 호출
            response = llm_with_tools.invoke(messages)
            
            # 도구 호출이 없으면 종료
            if not getattr(response, "tool_calls", None):
                break

            messages.append(response)  # LLM 응답 기록
            logger.info(f"응답 받음 - tool_calls: {len(response.tool_calls)}개")
            
            # 4. 도구 실행
            tool_messages = execute_tools(llm_with_tools, tools_dict, response)
            messages.extend(tool_messages)
            
            logger.info("🟢 messages:" + "\n\t".join(type(msg).__name__ for msg in messages))
            logger.info('='*60)
        
        return response.content
        
    except Exception as e:
        logger.error(f"🔴 오류 발생: {e}", exc_info=True)
        return f"오류가 발생했습니다: {str(e)}"

In [15]:
answer = ask_search('12월에 서울 축제 뭐 있어?')

# 깔끔하게 출력
print("\n🌟 12월 서울 축제 안내 🌟\n")
for line in answer.split("\n"):
    line = line.strip()
    if line:  # 빈 줄 제거
        print(line)
print("\n🎉 즐거운 겨울 축제 보내세요! 🎉")

2025-11-11 10:53:17,911 - INFO - 
2025-11-11 10:53:17,911 - INFO - 🔵 사용자 질문: 12월에 서울 축제 뭐 있어?
2025-11-11 10:53:17,912 - INFO - ============================================================
2025-11-11 10:53:17,919 - INFO - 검색 에이전트 설정 완료
2025-11-11 10:53:17,920 - INFO - 메시지 초기화 완료
2025-11-11 10:53:19,161 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-11 10:53:19,164 - INFO - 응답 받음 - tool_calls: 1개
2025-11-11 10:53:19,165 - INFO - 도구 호출 발견: 1개
2025-11-11 10:53:19,166 - INFO - 도구 실행 요청 ▶ google_search
2025-11-11 10:53:19,166 - INFO -   - ID: call_DFvoFDtmhSEt3WPQzVQh5ae0
2025-11-11 10:53:19,167 - INFO -   - Args: {'query': '12월 서울 축제'}
2025-11-11 10:53:19,171 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-11-11 10:53:20,165 - INFO - ✅ 도구 실행 결과: ... 서울의 공원 2023-12-01 ~ 2026-09-29; 축제&행사 행사일정 남산봉수의식 등 전통 ... 월~금, 오전 9시~오후 6시 staff@visitseoul.net. 문의: 여행자 지원센터 일반 ... 페스티벌 in 서울 페스티벌 in 서울. PrevNext. Start. Stop. 1. 2. 3. 4. 


🌟 12월 서울 축제 안내 🌟

12월에 서울에서 열리는 주요 축제들을 알려드릴게요.
1. 서울빛초롱축제
- 기간: 12월 13일(금) ~ 1월 12일(일)
- 장소: 청계천 일대 (청계광장~수표교)
- 내용: 환상적인 빛 조형물과 라이트쇼를 즐길 수 있는 겨울 축제
2. DDP 서울라이트
- 기간: 12월 중순부터 연말까지 개최 예정
- 장소: 동대문디자인플라자(DDP)
- 내용: 화려한 라이트쇼와 미디어 아트가 어우러지는 축제
3. 서울윈터페스타
- 기간: 12월 중순 예정
- 장소: 서울 도심 곳곳
- 내용: 겨울 분위기를 만끽할 수 있는 다채로운 행사와 축제
이 외에도 청계천 광교갤러리 등지에서 소망등 띄우기 같은 행사도 진행된다고 해요. 자세한 일정과 장소는 축제 공식 홈페이지나 서울시 관광 안내에서 확인하시면 좋습니다.

🎉 즐거운 겨울 축제 보내세요! 🎉
